In [ ]:
# Audio to Text Transcription with Groq
# This notebook converts audio files to MP3 and transcribes them using Groq's API

import os
import subprocess
from pathlib import Path
from groq import Groq
import tempfile

# Install required packages (run once)
# !pip install groq pydub

# Configuration
GROQ_API_KEY = "your_groq_api_key_here"  # Replace with your actual API key
INPUT_FOLDER = "audio_files"  # Folder containing your audio files
OUTPUT_FOLDER = "converted_mp3"  # Folder for converted MP3 files

def check_ffmpeg():
    """Check if FFmpeg is installed"""
    try:
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
        print("✓ FFmpeg is available")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("❌ FFmpeg not found. Please install FFmpeg:")
        print("- Windows: Download from https://ffmpeg.org/download.html")
        print("- macOS: brew install ffmpeg")
        print("- Linux: sudo apt install ffmpeg")
        return False

def convert_to_mp3(input_file, output_file):
    """Convert audio file to MP3 using FFmpeg"""
    try:
        cmd = [
            'ffmpeg',
            '-i', str(input_file),
            '-codec:a', 'libmp3lame',
            '-b:a', '192k',
            '-y',  # Overwrite output file if it exists
            str(output_file)
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print(f"✓ Converted: {input_file.name} -> {output_file.name}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Error converting {input_file.name}: {e.stderr}")
        return False

def transcribe_audio(audio_file_path, groq_client):
    """Transcribe audio file using Groq API"""
    try:
        with open(audio_file_path, "rb") as file:
            transcription = groq_client.audio.transcriptions.create(
                file=(audio_file_path.name, file.read()),
                model="whisper-large-v3",
                response_format="text"
            )
        return transcription
    except Exception as e:
        print(f"❌ Error transcribing {audio_file_path.name}: {str(e)}")
        return None

def process_audio_files(input_folder, output_folder, groq_api_key):
    """Main function to process all audio files"""
    
    # Check if FFmpeg is available
    if not check_ffmpeg():
        return
    
    # Initialize Groq client
    try:
        groq_client = Groq(api_key=groq_api_key)
        print("✓ Groq client initialized")
    except Exception as e:
        print(f"❌ Error initializing Groq client: {str(e)}")
        return
    
    # Create folders if they don't exist
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    output_path.mkdir(exist_ok=True)
    
    if not input_path.exists():
        print(f"❌ Input folder '{input_folder}' not found")
        return
    
    # Supported audio formats
    supported_formats = {'.ogg', '.wav', '.flac', '.m4a', '.aac', '.wma', '.mp3'}
    
    # Process each audio file
    audio_files = [f for f in input_path.iterdir() 
                  if f.is_file() and f.suffix.lower() in supported_formats]
    
    if not audio_files:
        print(f"❌ No supported audio files found in '{input_folder}'")
        return
    
    print(f"Found {len(audio_files)} audio files to process")
    print("-" * 50)
    
    transcriptions = {}
    
    for audio_file in audio_files:
        print(f"Processing: {audio_file.name}")
        
        # Check if file is already MP3
        if audio_file.suffix.lower() == '.mp3':
            mp3_file = audio_file
            print(f"✓ Already MP3: {audio_file.name}")
        else:
            # Convert to MP3
            mp3_file = output_path / f"{audio_file.stem}.mp3"
            if not convert_to_mp3(audio_file, mp3_file):
                continue
        
        # Transcribe the MP3 file
        print(f"🎤 Transcribing: {mp3_file.name}")
        transcription = transcribe_audio(mp3_file, groq_client)
        
        if transcription:
            transcriptions[audio_file.name] = transcription
            print(f"✓ Transcription completed for: {audio_file.name}")
            
            # Save transcription to text file
            txt_file = output_path / f"{audio_file.stem}_transcription.txt"
            with open(txt_file, 'w', encoding='utf-8') as f:
                f.write(transcription)
            print(f"✓ Saved transcription: {txt_file.name}")
        
        print("-" * 30)
    
    return transcriptions

# Example usage for single file conversion and transcription
def process_single_file(file_path, groq_api_key):
    """Process a single audio file"""
    file_path = Path(file_path)
    
    if not file_path.exists():
        print(f"❌ File not found: {file_path}")
        return None
    
    # Initialize Groq client
    groq_client = Groq(api_key=groq_api_key)
    
    # Convert to MP3 if needed
    if file_path.suffix.lower() != '.mp3':
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_mp3 = Path(temp_dir) / f"{file_path.stem}.mp3"
            if convert_to_mp3(file_path, temp_mp3):
                transcription = transcribe_audio(temp_mp3, groq_client)
            else:
                return None
    else:
        transcription = transcribe_audio(file_path, groq_client)
    
    return transcription

# Run the main processing function
if __name__ == "__main__":
    # Make sure to set your Groq API key
    if GROQ_API_KEY == "your_groq_api_key_here":
        print("⚠️  Please set your Groq API key in the GROQ_API_KEY variable")
    else:
        # Process all files in the input folder
        transcriptions = process_audio_files(INPUT_FOLDER, OUTPUT_FOLDER, GROQ_API_KEY)
        
        if transcriptions:
            print("\n" + "="*50)
            print("TRANSCRIPTION RESULTS:")
            print("="*50)
            for filename, text in transcriptions.items():
                print(f"\n📁 {filename}:")
                print("-" * 30)
                print(text[:500] + "..." if len(text) > 500 else text)

# Alternative: Process a single file
# transcription = process_single_file("path/to/your/audio/file.ogg", GROQ_API_KEY)
# if transcription:
#     print("Transcription:", transcription)